<div style="text-align: center;">
    <img src="logoucm.png" style="height: 250px;">
    <br/>
    <span style="color:#888"></span>
</div>





<font size="6"><center>***Metodos numéricos en finanzas***</center></font>

<font size="5"><center><span style="color:blue">***Nombre y Apellidos: Andrés Velasco Sánchez***</span></center></font>

## Importamos las librerías necesarias

In [528]:
import numpy as np
import pandas as pd
from math import sin, cos, sqrt, log, exp, pi
import matplotlib.pyplot as plt
import scipy as sp
from scipy.stats import t
import scipy.stats as si

In [529]:
m=2**(31)-1
a=16807
b=0

In [530]:
def imprimir_tabla_estadisticas(df):
    print("\n" + "="*75)
    print("      RESULTADOS MONTE CARLO – OPCIONES EUROPEAS")
    print("="*75)
    print(f"{'Opción':<10} | {'Valor estimado':>15} | {'Varianza':>15} | {'Precisión (95%)':>18}")
    print("-"*75)

    for _, row in df.iterrows():
        print(f"{row['Opción']:<10} | "
              f"{row['Valor estimado']:>15.6f} | "
              f"{row['Varianza']:>15.6f} | "
              f"{row['Precisión (95%)']:>18.6f}")

    print("="*75 + "\n")

In [531]:

def generador_congruencial(semilla, m, a, b):
    x = semilla
    inv_m = 1 / m
    while True:
        x = (a * x + b) % m
        yield x * inv_m
def box_muller(N,ant=False):
    g_2 = generador_congruencial(123456789, m, a, b)
    n = N * 2
    U = [1 - next(g_2) if ant else next(g_2) for _ in range(n)]
    u1 = np.array(U[0::2])
    u2 = np.array(U[1::2])

    z0 = np.sqrt(-2.0 * np.log(u1)) * np.cos(2.0 * np.pi * u2)
    z1 = np.sqrt(-2.0 * np.log(u1)) * np.sin(2.0 * np.pi * u2)
    return z0, z1


def marsaglia_method(N,ant=False):
    g_3 = generador_congruencial(123456789, m, a, b)
    X = []
    Y = []
    n = 0
    while n<N:
        U_1 = 1-next(g_3) if ant else next(g_3)
        U_2 = 1-next(g_3) if ant else next(g_3)
        v_1 = 2 * U_1 - 1
        v_2 = 2 * U_2 - 1
        R_2 = v_1**2 + v_2**2
        if R_2 >= 1:
            continue

        y = np.sqrt(-2 * np.log(R_2) / R_2)
        X.append(v_1 * y)
        Y.append(v_2 * y)
        n += 1

    return np.array(X), np.array(Y)

def brownian_motion(T, N,X,Y):
    dt = T / N
    X = [0]+X.tolist()
    Y = [0]+Y.tolist()
    X = np.cumsum(np.sqrt(dt) * np.array(X))
    Y = np.cumsum(np.sqrt(dt) * np.array(Y))
    return X, Y

<font size="6"><center><span style="color:blue">***Práctica 2***</span></center></font>

<font size="4" color=blue>**Apartado a**</font>
***

Obtener los valores de una opción call y una opción put europeas con vencimiento $T$. Dar la estimación del valor y la precisión si:  

$$
V_{\mathrm{call}} = e^{-r_c t} \mathbb{E}_{\mathrm{riesgo~neutro}} [\textrm{payoff}] = (1+r)^{-t} \mathbb{E}_{\mathrm{riesgo~neutro}}[\textrm{payoff}].
$$

- **(a.1.)** Obtener el valor de una opción call usando la fórmula de Black-Scholes:  

$$
V_{\mathrm{call}}(S,t) = Se^{-d_c t} \Phi(d_1) - Ke^{-r_c t} \Phi(d_2),
$$

donde:
$$
d_1 = \frac{\ln(S/K) + (r_c -d_c + 0.5 \sigma^2)t}{\sigma \sqrt{t}}, \quad d_2 = d_1 -\sigma\sqrt{t}.
$$

**Datos:**
- $r = 0.03$
- Strike = $14$
- Plazo $T = 1,2,3$
- $\sigma = 0.3$
- $S_0 = 14$
- $d = 0$

<font size="2">**Nota:** La varianza de la estimación del valor es $S^2_{\mathrm{call}} = (e^{-r_c t})^2 S^2_{\mathrm{\overline{payoff}}}$.
    
<font size="2" color=grey>**Nota para comprobación:** $t=1; V_{\mathrm{call}}=1.856787;V_{\mathrm{put}}=1.449020$.</font>



In [532]:
r = 0.03
r_c = log(1+r)
K = 14
T = 1
sigma = 0.3
S0 = 14
d_c = 0
d=0
N = 1000   


In [533]:
def black_scholes(S0, K, T, r_c, sigma, d_c):
    d1 = (np.log(S0/K) + (r_c - d_c + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    V_call = S0*si.norm.cdf(d1)-K*np.exp(-r_c*T)*si.norm.cdf(d2)
    V_put = K*np.exp(-r_c*T)*si.norm.cdf(-d2)-S0*si.norm.cdf(-d1)
    return V_call, V_put

V_call_1,V_put_1 = black_scholes(S0, K, T, r_c, sigma,d_c)

In [534]:
print("\n" + "="*72)
print("      VALORACIÓN BLACK-SCHOLES - OPCIONES EUROPEAS")
print("="*72)
print(f"{'T (años)':<10} | {'Call':>15} | {'Put':>15}")
print("-"*72)
plazos = [1, 2, 3]
for j in plazos:
    V_call, V_put = black_scholes(S0, K, j, r_c, sigma, d_c)
    print(f"{j:<10} | {V_call:>15.6f} | {V_put:>15.6f}")

print("="*72 + "\n")


      VALORACIÓN BLACK-SCHOLES - OPCIONES EUROPEAS
T (años)   |            Call |             Put
------------------------------------------------------------------------
1          |        1.856787 |        1.449020
2          |        2.708070 |        1.904413
3          |        3.381072 |        2.193055



In [535]:
assert f"{V_call_1:.6f}" == "1.856787"
assert f"{V_put_1:.6f}" == "1.449020"

<font size="4" color=blue>**Apartado b**</font>
***
Obtener los valores de una opción call y una opción put europeas con vencimiento $T = 1$ (con los mismos datos del apartado anterior). Dar la estimación del valor y la precisión si:  

$$
V_{\mathrm{call}} = e^{-r_c t} \mathbb{E}_{\mathrm{riesgo~neutro}}[\textrm{payoff}] = (1+r)^{-t} \mathbb{E}_{\mathrm{riesgo~neutro}}[\textrm{payoff}].
$$

- **(b.1.)** Obtener el valor final del subyacente mediante la expresión en el plazo final:  

$$
S(t) = S(0) e^{(r_c - d_c - 0.5\sigma^2)t + \sigma \sqrt{t}\mathcal{N}(0,1)}.
$$

<font size="2" color=grey>**Nota para comprobación:** $t=1; V_{\mathrm{call}}=1.919841;Varianza=10.065818;Precisión=0.196879$.</font>

In [536]:
  
Z0,Z1 = box_muller(N)

S_T=S0*np.exp((r_c-d_c-0.5*sigma**2)*T+sigma*np.sqrt(T)*Z1)


V_call_vals = exp(-r_c*T) * np.maximum(S_T-K, 0)
V_put_vals = exp(-r_c*T) * np.maximum(K-S_T, 0)



def calcular_estadisticas_montecarlo(valores):
    n_muestras = len(valores)
    media = np.mean(valores)
    varianza = np.var(valores, ddof=1)
    desviacion_std = np.std(valores, ddof=1)
    t_critico = t.ppf(0.975, df=n_muestras - 1)

    precision = t_critico * (desviacion_std / sqrt(n_muestras))

    return media, varianza, precision

media_call, var_call, prec_call = calcular_estadisticas_montecarlo(V_call_vals)
media_put,  var_put,  prec_put  = calcular_estadisticas_montecarlo(V_put_vals)


In [537]:
tabla_resultados = pd.DataFrame([
    {
        "Opción": "Call",
        "Valor estimado": media_call,
        "Varianza": var_call,
        "Precisión (95%)": prec_call
    },
    {
        "Opción": "Put",
        "Valor estimado": media_put,
        "Varianza": var_put,
        "Precisión (95%)": prec_put
    }
])

imprimir_tabla_estadisticas(tabla_resultados)


      RESULTADOS MONTE CARLO – OPCIONES EUROPEAS
Opción     |  Valor estimado |        Varianza |    Precisión (95%)
---------------------------------------------------------------------------
Call       |        1.919841 |       10.065818 |           0.196879
Put        |        1.414743 |        3.615416 |           0.117992



In [538]:
assert f"{media_call:.6f}"== '1.919841'
assert f"{var_call:.6f}"== '10.065818'
assert f"{prec_call:.6f}"== '0.196879'


    
- **(b.2.)** Obtener el valor final del subyacente mediante simulación dinámica con $\Delta t = 1/12$ y la expresión (Lema de Itô):  

$$
S \leftarrow S e^{(r_c - d_c - 0.5\sigma^2)\Delta t + \sigma \sqrt{\Delta t}\mathcal{N}(0,1)}.
$$

<font size="2" color=grey>**Nota para comprobación:** $t=1; V_{\mathrm{call}}=2.008849;Varianza=10.349455;Precisión=0.199633$.</font>

In [539]:

delta_t = 1/12
N_caminos = 1000
M = int(T/delta_t)

In [540]:
def simular_caminos_browniano(S0, r_c, d_c, sigma, T,Z_n, dt=1/12, N_caminos=100):
    M = int(T / dt)  
    Z_n = Z_n.reshape(N_caminos, M)

    log_ST=np.log(S0)+np.sum((r_c-d_c-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*Z_n, axis=1)
    ST=np.exp(log_ST)

    return ST




In [541]:


Z0, Z1 = box_muller(N_caminos * M)  

S_T = simular_caminos_browniano(S0, r_c, d_c, sigma, T, Z1,N_caminos=1000)
V_call_vals = exp(-r_c*T) * np.maximum(S_T-K, 0)
V_put_vals = exp(-r_c*T) * np.maximum(K-S_T, 0)


media_call_lema, var_call_lema, prec_call_lema = calcular_estadisticas_montecarlo(V_call_vals)
media_put_lema,  var_put_lema,  prec_put_lema  = calcular_estadisticas_montecarlo(V_put_vals)



In [542]:

tabla_resultados = pd.DataFrame([
    {
        "Opción": "Call",
        "Valor estimado": media_call_lema,
        "Varianza": var_call_lema,
        "Precisión (95%)": prec_call_lema
    },
    {
        "Opción": "Put",
        "Valor estimado": media_put_lema,
        "Varianza": var_put_lema,
        "Precisión (95%)": prec_put_lema
    }
])

imprimir_tabla_estadisticas(tabla_resultados)


      RESULTADOS MONTE CARLO – OPCIONES EUROPEAS
Opción     |  Valor estimado |        Varianza |    Precisión (95%)
---------------------------------------------------------------------------
Call       |        2.008849 |       10.349455 |           0.199633
Put        |        1.388021 |        3.521192 |           0.116445



In [543]:
assert f"{media_call_lema:.6f}"== '2.008849'
assert f"{var_call_lema:.6f}"== '10.349455'
assert f"{prec_call_lema:.6f}"== '0.199633'

- **(b.3.)** Obtener el valor final del subyacente mediante simulación dinámica con $\Delta t = 1/12$ y el mecanismo de transición. (Ecuación estocástica)  

$$
S \leftarrow S + S(r_{\Delta t} - d_{\Delta t})+\sigma S \sqrt{\Delta t}\mathcal{N}(0,1) = S(1+r-d)^{\Delta t} + \sigma S \sqrt{\Delta t} \mathcal{N}(0,1)
$$
    

<font size="2" color=grey>**Nota para comprobación:** $t=1; V_{\mathrm{call}}=2.005745;Varianza=10.020386;Precisión=0.196434$.</font>

In [544]:
def ecuacion_estocastica(S0, r, d, sigma, T,Z_n, dt=1/12, N_caminos=100):
    M = int(T / dt)  
    Z_n = Z_n.reshape(N_caminos, M)
    multipliers=(1+r-d)**dt+sigma*np.sqrt(dt)*Z_n

    ST= S0*np.prod(multipliers, axis=1)

    return ST







In [545]:
Z0, Z1 = box_muller(N_caminos * M) 

S_T = ecuacion_estocastica(S0, r, d, sigma, T, Z1,N_caminos = 1000)

V_call_vals = exp(-r_c*T) * np.maximum(S_T-K, 0)
V_put_vals = exp(-r_c*T) * np.maximum(K-S_T, 0)


media_call_estc, var_call_estc, prec_call_estc = calcular_estadisticas_montecarlo(V_call_vals)
media_put_estc,  var_put_estc,  prec_put_estc  = calcular_estadisticas_montecarlo(V_put_vals)



In [546]:
tabla_resultados = pd.DataFrame([
    {
        "Opción": "Call",
        "Valor estimado": media_call_estc,
        "Varianza": var_call_estc,
        "Precisión (95%)": prec_call_estc
    },
    {
        "Opción": "Put",
        "Valor estimado": media_put_estc,
        "Varianza": var_put_estc,
        "Precisión (95%)": prec_put_estc
    }
])


imprimir_tabla_estadisticas(tabla_resultados)


      RESULTADOS MONTE CARLO – OPCIONES EUROPEAS
Opción     |  Valor estimado |        Varianza |    Precisión (95%)
---------------------------------------------------------------------------
Call       |        2.005745 |       10.020386 |           0.196434
Put        |        1.392958 |        3.616090 |           0.118003



In [547]:
assert f"{media_call_estc:.6f}"== '2.005745'
assert f"{var_call_estc:.6f}"== '10.020386'
assert f"{prec_call_estc:.6f}"== '0.196434'

    
- **(b.4.)** Usando el valor final del subyacente mediante la expresión en el plazo final, determinar el tamaño muestral para que la estimación sea de un céntimo de Euro.

<font size="2" color=grey>**Nota para comprobación:** $t=1;n \geq 300mil$.</font>

In [548]:
      
precision_deseada = 0.01 
  


Z = box_muller(N_caminos)
Z = Z[1]
ST = S0 * np.exp((r_c - d_c - 0.5*sigma**2)*T + sigma*np.sqrt(T)*Z)

vals_call=np.exp(-r_c*T)*np.maximum(S_T-K, 0)
desviacion_std=np.std(vals_call, ddof=1)
varianza=np.var(vals_call, ddof=1)

precision_deseada=0.01
valor_critico=t.ppf(0.975, N_caminos-1) 

numero_sim=(valor_critico*desviacion_std/precision_deseada)**2
numero_sim_final=int(np.ceil(numero_sim))

print(f"Tamaño muestral necesario para precisión de 0.01 €: {numero_sim_final}")


Tamaño muestral necesario para precisión de 0.01 €: 385864


In [549]:
assert numero_sim_final >= 300000